In [26]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold,GridSearchCV
import lightgbm as lgb




In [13]:
train_df = pd.read_csv("./data/mushroom_train.csv", encoding="utf-8")
test_df = pd.read_csv("./data/mushroom_test.csv", encoding="utf-8")

In [14]:
df = pd.concat([train_df, test_df]).reset_index(drop=True)

In [15]:
df

,mushroom_id,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,0,p,x,s,n,t,p,f,c,n,...,s,w,w,p,w,o,p,k,s,u
1,1,e,x,s,y,t,a,f,c,b,...,s,w,w,p,w,o,p,n,n,g
2,2,e,b,s,w,t,l,f,c,b,...,s,w,w,p,w,o,p,n,n,m
3,3,p,x,y,w,t,p,f,c,n,...,s,w,w,p,w,o,p,k,s,u
4,4,e,x,s,g,f,n,f,w,b,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,8119,NaN,k,s,n,f,n,a,c,b,...,s,o,o,p,o,o,p,b,c,l
8120,8120,NaN,x,s,n,f,n,a,c,b,...,s,o,o,p,n,o,p,b,v,l
8121,8121,NaN,f,s,n,f,n,a,c,b,...,s,o,o,p,o,o,p,b,c,l
8122,8122,NaN,k,y,n,f,y,f,c,n,...,k,w,w,p,w,o,e,w,v,l


In [16]:
# label encoder
# class : { 0 : e , 1 : p , 2 : nan}

label_encoder = LabelEncoder()

name_columns = ['class','cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat']

for name in name_columns:
    df[name] = label_encoder.fit_transform(df[name])
df

,mushroom_id,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,0,1,5,2,4,1,6,1,0,1,...,2,7,7,0,2,1,4,2,3,5
1,1,0,5,2,9,1,0,1,0,0,...,2,7,7,0,2,1,4,3,2,1
2,2,0,0,2,8,1,3,1,0,0,...,2,7,7,0,2,1,4,3,2,3
3,3,1,5,3,8,1,6,1,0,1,...,2,7,7,0,2,1,4,2,3,5
4,4,0,5,2,3,0,5,1,1,0,...,2,7,7,0,2,1,0,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,8119,2,3,2,4,0,5,0,0,0,...,2,5,5,0,1,1,4,0,1,2
8120,8120,2,5,2,4,0,5,0,0,0,...,2,5,5,0,0,1,4,0,4,2
8121,8121,2,2,2,4,0,5,0,0,0,...,2,5,5,0,1,1,4,0,1,2
8122,8122,2,3,3,4,0,8,1,0,1,...,1,7,7,0,2,1,0,7,4,2


In [17]:
train_df = df[:6500]
test_df = df[6500:]

In [19]:
test_df = test_df.drop(["class"],axis=1)
test_df

,mushroom_id,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
6500,6500,2,2,4,0,2,1,0,1,0,...,2,6,6,0,2,1,0,7,4,0
6501,6501,3,3,2,0,7,1,0,1,0,...,2,6,6,0,2,1,0,7,4,0
6502,6502,2,3,2,0,2,1,0,1,0,...,1,7,7,0,2,1,0,7,4,0
6503,6503,2,2,2,0,2,1,0,1,0,...,1,7,6,0,2,1,0,7,4,0
6504,6504,2,3,2,0,8,1,0,1,0,...,2,7,6,0,2,1,0,7,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,8119,3,2,4,0,5,0,0,0,11,...,2,5,5,0,1,1,4,0,1,2
8120,8120,5,2,4,0,5,0,0,0,11,...,2,5,5,0,0,1,4,0,4,2
8121,8121,2,2,4,0,5,0,0,0,5,...,2,5,5,0,1,1,4,0,1,2
8122,8122,3,3,4,0,8,1,0,1,0,...,1,7,7,0,2,1,0,7,4,2


In [20]:
x_train = train_df.drop(['mushroom_id','class'],axis=1)
y_train = train_df['class']
x_test = test_df.drop(['mushroom_id'],axis=1)

In [28]:
model = lgb.LGBMClassifier()

stratified_kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=1234)

param_grid = {
    'n_estimators' : np.arange(1,100,10),
    "max_depth":np.arange(1,10,1)
}

grid_search = GridSearchCV(
    estimator = model,
    param_grid = param_grid,
    cv = stratified_kf
)

In [29]:
result = grid_search.fit(x_train,y_train)

[LightGBM] [Info] Number of positive: 2239, number of negative: 2961
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000299 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 100
[LightGBM] [Info] Number of data points in the train set: 5200, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.430577 -> initscore=-0.279498
[LightGBM] [Info] Start training from score -0.279498
[LightGBM] [Info] Number of positive: 2239, number of negative: 2961
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000224 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 101
[LightGBM] [Info] Number of data points in the train set: 5200, number of used features: 19
[LightGBM] [Info] [binary:

In [31]:
result.best_params_

{'max_depth': 4, 'n_estimators': 51}

In [32]:
result.best_score_

1.0

In [33]:
model = lgb.LGBMClassifier(
    n_estimators=51,
    max_depth=4
)

In [34]:
model.fit(x_train,y_train)

[LightGBM] [Info] Number of positive: 2799, number of negative: 3701
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000267 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 101
[LightGBM] [Info] Number of data points in the train set: 6500, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.430615 -> initscore=-0.279341
[LightGBM] [Info] Start training from score -0.279341
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

LGBMClassifier(max_depth=4, n_estimators=51)

In [35]:
model.score(x_train,y_train)

1.0

In [36]:
y_test_pred = model.predict(x_test)

In [37]:
test_df['class'] = y_test_pred

In [39]:
test_df["class"] = test_df["class"].replace([1,0],["p","e"])

In [40]:
test_df[['mushroom_id','class']].to_csv('./data/mushroom_submission_1.csv',index=False)